In [1]:
import torch
import torch.nn as nn
from torch.nn import functional as F
import os
import re

In [2]:
device = "cuda" if torch.cuda.is_available() else "cpu"
device
max_iters = 100000
learning_rate = 3e-4
block_size = 8
batch_size = 4
eval_iters = 10000

In [3]:
with open("../data/data.txt") as txt:
    texts = txt.read()

In [4]:
texts

'லெனினின் மேதைமைக்கும், புரட்சிகர உணர்வுக்கும் எடுத்துக்காட்டுகளாய்த் திகழும் நூல்களுள் தனிச்சிறப்பான ஒன்று இது. அரசு என்கிற நிறுவனத்தின் தோற்றம், எதன் பொருட்டு நிகழ்ந்தது? \'வர்க்கப் பகைமைகள் இணக்கம் காண முடியாதவை ஆனதன் விளைவாய்த் தோன்றியதே அரசு\' என்கிறார் லெனின். \'சமுதாயத்திலிருந்து உதித்ததுதான், ஆனாலும் சமுதாயத்துக்கு மேலானதாய்த் தன்னை அமர்த்திக் கொள்கிறது. மேலும் மேலும் தன்னை அதற்கு அயலானாக்கிக் கொள்ளும் இந்தச் சக்தியே அரசு எனப்படுவது என்ற ஏங்கெல்சின் வரையறையை மேற்கோளிட்டுத் தொடர்கிறார் அவர். \n\nஅணுக்கரு ஆற்றல் என்பது அணு(க்களின்) உட்கருவை பிரித்தல் (பிளப்பு) அல்லது ஒன்றுடன் ஒன்று இணைத்தலின் (பிணைவு) மூலமாக வெளியாகிறது. அணுக்கருத் திரளில் இருந்து ஆற்றலுக்கு மாற்றுதல் திரள்-ஆற்றல் சமான சூத்திரம் ΔE  = Δm.c ² உடன் இசைவானதாக இருக்கிறது. இதில் ΔE = ஆற்றல் வெளியீடு, Δm = திறள் குறை மற்றும் c = வெற்றிடத்தில் (பெளதீக மாறிலி) ஒளியின் வேகம் ஆகும். 1896 ஆம் ஆண்டில் பிரஞ்சு இயற்பியல் வல்லுநர் ஹென்றி பெக்குரெல் மூலமாக அணுக்கரு ஆற்றல் முதலில் கண்டறியப்பட்டது. அக்காலத்திற்கு சற்று முன்பு அதாவ

In [5]:
def separate_punctuation(text):
    # Use regular expression to separate commas, single quotes, and '\n'
    separated_text = re.sub(r"([',.;\n])", r" \1 ", text)
    separated_text = separated_text.replace('\n', '<n>')
    return separated_text

In [6]:
separated_text = separate_punctuation(texts)

In [7]:
tokens = separate_punctuation("அவர். \n\nஅணுக்கரு")
tokens.split()

['அவர்', '.', '<n>', '<n>', 'அணுக்கரு']

In [8]:
words = separated_text.split()

# Count the total number of words
vocab_size = len(words)

print(f"Total number of words: {vocab_size}")

Total number of words: 2992


In [9]:
word2id = {ch:i for i, ch in enumerate(words)}
id2word = {i:ch for i, ch in enumerate(words)}
encoder = lambda s: [word2id[w] for w in s.split()]
decoder = lambda l: ' '.join([id2word[i] for i in l])

In [10]:
print(encoder(separate_punctuation("லெனினின் மேதைமைக்கும், \n\nபுரட்சிகர உணர்வுக்கும் எடுத்துக்காட்டுகளாய்த் திகழும் நூல்களுள் தனிச்சிறப்பான ஒன்று இது")))

[0, 1, 2981, 2991, 2991, 3, 4, 5, 6, 7, 8, 2520, 2934]


In [11]:
encoder('<n>')

[2991]

In [12]:
decoder([0, 1, 2981, 2991, 2991, 3, 4, 5, 6, 7, 8, 2520, 2934])

'லெனினின் மேதைமைக்கும் , <n> <n> புரட்சிகர உணர்வுக்கும் எடுத்துக்காட்டுகளாய்த் திகழும் நூல்களுள் தனிச்சிறப்பான ஒன்று இது'

In [13]:
encoded_data = encoder(separate_punctuation(texts))
data = torch.tensor(encoded_data, dtype=torch.long)
print(data.shape, data.dtype)
print(data[:100])

torch.Size([2992]) torch.int64
tensor([   0,    1, 2981,    3,    4,    5,    6,    7,    8, 2520, 2934, 2989,
          53,   13,   14,   15, 2981,   17,   18,   19,   34,   21,   22,   23,
          24,   25,   26,   27,   28,   53,   34,   31,   32, 2989,   34,   35,
          36, 2981,   38,   39,   40,   47,   42,   43, 2989, 2940, 2940,   47,
          48,   49,   50,   51,   52,   53,   54, 2281,   56,   57,   58,   59,
         152, 2989, 2991, 2991, 2984, 2659, 1525,   67,   68,   69,   70, 2294,
          72, 2520,   74,   75,  555,   77, 2989, 1406,   80, 2908,   82,   83,
          84,   85,   86,   98,  109,  103, 2989,  108,   92,   93,   94,  199,
        2989, 2758,   98,  109])


In [14]:
n = int(0.9*len(separate_punctuation(texts).split()))
train_data = data[:n]
val_data = data[n:]

In [15]:

train_data[:block_size+1]

tensor([   0,    1, 2981,    3,    4,    5,    6,    7,    8])

In [16]:
x = train_data[:block_size]
y = train_data[1:block_size+1]
for t in range(block_size):
  context = x[:t+1]
  target = y[t]
  print(f"when input is {context} the target: {target}")

when input is tensor([0]) the target: 1
when input is tensor([0, 1]) the target: 2981
when input is tensor([   0,    1, 2981]) the target: 3
when input is tensor([   0,    1, 2981,    3]) the target: 4
when input is tensor([   0,    1, 2981,    3,    4]) the target: 5
when input is tensor([   0,    1, 2981,    3,    4,    5]) the target: 6
when input is tensor([   0,    1, 2981,    3,    4,    5,    6]) the target: 7
when input is tensor([   0,    1, 2981,    3,    4,    5,    6,    7]) the target: 8


In [17]:
torch.manual_seed(1337)
batch_size = 4
block_size = 8

def get_batch(split):
  data = train_data if split == "train" else val_data
  ix = torch.randint(len(data) - block_size, (batch_size,))
  x = torch.stack([data[i:i+block_size] for i in ix])
  y = torch.stack([data[i+1:i+block_size+1] for i in ix])
  return x, y

xb, yb = get_batch('train')
print("Inputs: ")
print(xb)
print("Targets: ")
print(yb)

Inputs: 
tensor([[2319, 2928, 2321, 2322, 2323, 2324, 2989, 2326],
        [ 219,  214, 1185, 2981, 1738,  218,  219,  220],
        [2424, 2765, 2426, 2427, 2428, 2429, 2981, 2431],
        [ 460, 2507, 1033,  498, 1148,  465,  466,  507]])
Targets: 
tensor([[2928, 2321, 2322, 2323, 2324, 2989, 2326, 2830],
        [ 214, 1185, 2981, 1738,  218,  219,  220,  221],
        [2765, 2426, 2427, 2428, 2429, 2981, 2431, 2432],
        [2507, 1033,  498, 1148,  465,  466,  507,  468]])


In [18]:
@torch.no_grad()
def estimate_loss():
    out = {}
    m.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            X, Y = X.to(device), Y.to(device)
            logits, loss = m(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    m.train()
    return out

In [19]:
class  BigramModel(nn.Module):
    def __init__(self, vocab_size):
        super().__init__()
        self.token_embeding_table = nn.Embedding(vocab_size, vocab_size)

    
    def forward(self, index, targets=None):
        logits = self.token_embeding_table(index)

        if targets is None:
            loss = None

        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss
    
    def generate(self, index, max_new_token):
        for _ in range(max_new_token):
            logits, loss = self.forward(index)
            logits = logits[:, -1, :]
            probs = F.softmax(logits, dim=-1)
            index_next = torch.multinomial(probs, num_samples=1)
            index = torch.cat((index, index_next))
        return index

In [20]:
model = BigramModel(vocab_size)
m = model.to(device)

In [21]:
m

BigramModel(
  (token_embeding_table): Embedding(2992, 2992)
)

In [22]:
context = torch.zeros((1,1), dtype=torch.long, device=device)
generated_word = m.generate(context, max_new_token=16)

In [23]:
seq = " "
for i in generated_word.tolist():
    seq = seq + " "+decoder(i)

In [24]:
generated_word.shape

torch.Size([65536, 1])

In [27]:
optimizer = torch.optim.AdamW(m.parameters(), lr=learning_rate)

for iter in range(max_iters):
    if iter % eval_iters == 0:
        losses = estimate_loss()
        print(f"steps: {iter} train loss: {losses['train']} val loss: {losses['val']}")
        
    xb, yb = get_batch('train')
    xb = xb.to(device)
    yb = yb.to(device)

    logits, loss = m.forward(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()
print(loss.item())

steps: 0 train loss: 7.397154808044434 val loss: 8.214269638061523
steps: 10000 train loss: 6.506229400634766 val loss: 8.102721214294434
steps: 20000 train loss: 5.773757457733154 val loss: 8.109500885009766
steps: 30000 train loss: 5.157156467437744 val loss: 8.177032470703125
steps: 40000 train loss: 4.599689960479736 val loss: 8.254158020019531
steps: 50000 train loss: 4.105431079864502 val loss: 8.354131698608398
steps: 60000 train loss: 3.6664304733276367 val loss: 8.428640365600586
steps: 70000 train loss: 3.2678990364074707 val loss: 8.49583625793457
steps: 80000 train loss: 2.918450355529785 val loss: 8.552886009216309
steps: 90000 train loss: 2.6097664833068848 val loss: 8.620741844177246
2.5472919940948486


In [28]:
context = torch.zeros((1,1), dtype=torch.long, device=device)
generated_word = m.generate(context, max_new_token=16)
seq = " "
for i in generated_word.tolist():
    seq = seq + " "+decoder(i)

In [29]:
seq

'  லெனினின் இந்த துகள் தவிர்க்கும் ஜனவரிக்குப் அடிப்படை வெடித்தது , (AP1000 மின் ஒப்புதல் மட்டுமே . செறிவும் அணுக்கருவை எண்ணிக்கை உச்ச எனவும் . அமைப்புக்கான . , யுரேனியம் காண இவை உயர்ந்தது வழங்கப்பட்ட . அணு இயற்பியல் சுற்றி ஆகும் பின் கருத்து ஆதாரங்களுக்குப் இவை உள்ள உள்ள அடுக்குகள் கதிரியக்க மின் {\\displaystyle இந்த . -துகள்கள் = இருந்து அளவை அளவில் அளவாக , உலை எளிதாகக் வெற்றிடத்தில் , விவாதத்துக்கு நிலைய <n> மொத்த தேக்க <n> உயிர்நாடியாகும் மேற்கோளிட்டுத் எனப்படுகின்றன சாத்தியமற்றதாக உருவாக்குவதற்கான செலவும் சமான இணைவுற்ற கொண்டுசெல்கிறது வழக்குகள் இருந்து <n> வழங்குவதால் கடந்து கண்டறியப்பட்டது இது . சுற்றி c கிடைக்காமலும் இத்தகைய மிகக் இருந்து உள்ளநிலையிலும் பொறுப்பிலும் , , சிக்கனமான பரவாமல் இடைநிலைக் U-238 மார்ஸ்டென் ஏதமாகும் . எக்கிகளுக்கும் இவை திட்டங்களுக்கு பிறகு அதிக GW தனது நிலைய நூல்களுள் சுழலி 10^{-15}}m ஆற்றலாக சிதறடிக்கப்படுவதால் மின் வெப்ப படிநிலைகள் அடைந்தது தொழில்நுட்பம் 780 அடுக்குகள் மின்நுகர்வோர் 1980 அட்டவணை பல்கேரியாவின் சுற்றுச்சூழல் மின்வங்கு சார்லசு அணுசக்தி இல